# Data Visualization using Python
### Muluken Muche
This data visualization use several python libraries to showcase available plots and charts to visualize publicly available NCSES data. The libraries include  matplotlib, seaborn, bokeh, plotly, ggplot, altair, and geoplotlib as necessary. The purpose is to create a template of figures so that we could use for similar data. Python codes are adopted from several freely available websites. These figures and charts  are based on publicly available NCSES data.   

In [5]:
# Libraries
%matplotlib inline
import os
import sys
import openpyxl
import pygal
import pyodbc
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from statsmodels.graphics import mosaicplot
from statsmodels.graphics.mosaicplot import mosaic
sns.set(style="ticks", color_codes=True)
import xlrd
import warnings
warnings.filterwarnings("ignore")
mpl.style.use('ggplot')

In [6]:
from IPython.display import display, HTML

base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

## 1) Doctorate Recipients of Science &E and Non-S&E by Broad and Detailed Fields (1958 - 2018)

The data is downloaded fromData Download from NCSES Interactive Data Tool. 				
https://ncses.norc.org/NSFTabEngine/#TABULATION. 

In [7]:
# Reading data from SQL Server in local machine
conn = pyodbc.connect("Driver={SQL Server};"
                      "Server=DESKTOP-USBNA55;"
                      "Database=SED;"
                      "Trusted_Connection=yes;"
)   

cursor = conn.cursor()

df1 = pd.read_sql_query('SELECT * from dbo.SED',conn)
df1 = df1.set_index(['S&E_Fields', 'Broad_Fields', 'Detailed_Fields'])
df1.head(3)
#df1.columns
#df1.dtypes

Empty DataFrame
Columns: [Column 0, Race_and_Ethnicity, Sex, Year, Number]
Index: []

In [11]:
# Data wrangling for proper formatting of visualizatiom
df1s = df1.stack()
df1s =df1s.to_frame()
df1s.head(10)
df1s = df1s.reset_index()
df1s.columns = ['S&E_Fields', 'Broad_Fields', 'Detailed_Fields', 'Year', 'Number']

df1s.head(3)
#df1s.shape

Empty DataFrame
Columns: [S&E_Fields, Broad_Fields, Detailed_Fields, Year, Number]
Index: []

In [10]:
df1s = df1s[df1s['Detailed Fields'] != 'Total']
df1s = df1s.reset_index()
df1s = df1s[['S&E Fields', 'Broad Fields', 'Detailed Fields', 'Year', 'Number']]
df1s.head(3)

# df1s.dtypes

KeyError: 'Detailed Fields'

### 1.1)   Bar Plots
The following stacked bar chart shows Science and Engineering versus Non-Science and Engineering doctorate recipients from 1958 to 2018. We can see that the magnitude of Science and Engineering doctorate recipients were increasing compared to the non-S&E. 

In [ ]:
p1 =  df1s.groupby(['S&E Fields', 'Year'])['Number'].sum()
p1 = p1.reset_index()
p1.head(20)
p1=p1.pivot(index='Year', columns='S&E Fields', values='Number')
p1.head(40)
p1.shape
# Adding the stacked=True option to plot() 
# creates a stacked bar plot
p1.plot(kind='barh',colormap='PRGn', figsize=(8, 10), width=1.0, stacked=True).legend(
    loc='lower right', ncol=1, title="S&E Fields ")
plt.title("Number of Doctorate Reciepients by Year (S&E and Non-S&E)")
plt.xlabel("Number of Doctorate Reciepients")
plt.ylabel("Year")

The following chart is just the above one in different way.

In [ ]:
df1.head(3)
SE_NSE = df1.reset_index()
SE_NSE = SE_NSE[SE_NSE['Broad Fields'] == 'Total']

SE_NSE.head()

SE_NSE_T = SE_NSE.T  # Transpose
SE_NSE_T = SE_NSE_T .reset_index()  # setting numerical index
SE_NSE_T.columns = SE_NSE_T.iloc[0] # Setting the first row as a column
SE_NSE_T = SE_NSE_T.set_index(['S&E Fields']) # setting S&E Fields column as an index
SE_NSE_T = SE_NSE_T.drop(['S&E Fields','Broad Fields', 'Detailed Fields'])  # dropping the first two rows 
                       #since it is created in the process but unnecessary
SE_NSE_T.head()
SE_NSE_T.plot(kind="bar",colormap='bwr', figsize=(14, 6))
plt.title("Doctorate Recipients of Science and Engineering and None-Science and Engineering (1958 - 2018) ")
plt.xlabel("Year")
plt.ylabel("Number of Doctorate Recipients")

The chart below shows doctorate recipients by broad fields both Science and Engineering (Engineering,Life Science, Mathematics and Computer Sciences, Physical Sciences and Earth Sciences, Psychology and Social Sciences) and Non-Science and Engineering (Education, Humanities and Arts, and Other non-S&E) from 1958 to 2018. We can see that Engineering and Life Sciences doctrate recipients are are relatively increasing in recent years where as Education doctrate recipients are somewhat decreasing in recent years.

In [ ]:
p2 =  df1s.groupby(['Broad Fields', 'Year'])['Number'].sum()
p2 = p2.reset_index()
p2=p2.pivot(index='Year', columns='Broad Fields', values='Number')


# Adding the stacked=True option to plot() 
# creates a stacked bar plot
p2.plot(kind='barh',colormap='tab10',figsize=(10, 12), width=1.0, stacked=True).legend(
    loc='lower right', ncol=1, title="Broad Fields ")
plt.title("Number of Doctorate Reciepients by Broad Fields and Year")
plt.xlabel("Number of Doctorate Reciepients")
plt.ylabel("Year")
#p2.info()

The bars below show doctorate recipients by detailed fields studies in both Science and Engineering  and Non-Science and Engineering from 1958 to 2018. Similar colors with different intensities show each broad fields in the above chart. Biological and biomedical sciences doctorates were the most increasing field of study especially in the last 12 years.

In [ ]:
p3 =  df1s.groupby(['Detailed Fields', 'Year'])['Number'].sum()
p3 = p3.reset_index()
p3=p3.pivot(index='Year', columns='Detailed Fields', values='Number')


# Adding the stacked=True option to plot() 
# creates a stacked bar plot
p3.plot(kind='barh',colormap='tab20b',figsize=(15, 18), width=1.0, stacked=True).legend(
    loc='lower right', ncol=1, title="Detailed Fields ")
plt.title("Number of Doctrate Reciepients by Detailed Fields and Year")
plt.xlabel("Number")
plt.ylabel("Year")
#plt.legend(title="Line", loc='upper left', handles=handles[::-1])

In [ ]:
# df1s['S&E Fields'].unique()
# df1s.count()
# df1s['Broad Fields'].unique()
# df = df1s.groupby('Broad Fields')['Number'].nunique()
# df.count()

# df1s['Detailed Fields'].unique()
# df = df1s.groupby('Detailed Fields')['Number'].nunique()
# df.count()

# df1s['Year'].unique()
# df = df1s.groupby('Year')['Number'].nunique()
# df.count()

In [ ]:
df1s.head()
SE = df1s[df1s['S&E Fields'] == 'Science and engineering']
SE.head()

sns.catplot(y="Broad Fields", x="Number", kind="box",height=3, aspect=3,
            data=SE).set(title="Number of Doctorate Recipients in Science & Engineering by Broad Fields (1958 - 2018) ");
sns.catplot(y="Detailed Fields", x="Number", kind="box",height=5, aspect=2,
            data=SE).set(title="Number of Doctorate Recipients in Science & Engineering by Detailed Fields (1958 - 2018) ");



#plt.annotate('Note: Data include 8 major field of studies', (0,0), (0, -40), xycoords='axes fraction', textcoords='offset points', va='top')


In [ ]:
df1s.head()
NSE = df1s[df1s['S&E Fields'] != 'Science and engineering']
NSE.head()
sns.catplot(y="Broad Fields", x="Number", kind="box",height=3, aspect=3,
            data=NSE).set(title="Number of Doctorate Recipients in Non-Science & Engineering by Broad Fields (1958 - 2018) ");
sns.catplot(y="Detailed Fields", x="Number", kind="box",height=5, aspect=2,
            data=NSE).set(title="Number of Doctorate Recipients in Non-Science & Engineering by Detailed Fields (1958 - 2018) ");

In [ ]:
SE_NSE_T = SE_NSE_T .reset_index()  # setting numerical index
SE_NSE_T = SE_NSE_T[['S&E Fields','Science and engineering','Non-science and engineering']]
SE_NSE_T.columns = [ 'Year', 'Science and engineering','Non-science and engineering']
SE_NSE_T = SE_NSE_T.set_index(['Year']) # setting S&E Fields column as an index
#SE_NSE_T = SE_NSE_T.drop(['S&E Fields','Broad Fields', 'Detailed Fields'])  # dropping the first two rows 
SE_NSE_TT = SE_NSE_T.T
# SE_NSE_TT.columns = SE_NSE_TT.iloc[0] # Setting the first row as a column
# SE_NSE_TT = SE_NSE_TT.set_index(['S&E Fields']) # setting S&E Fields column as an index
SE_NSE_TT.head()
#SE_NSE_T.head()
se = tuple(list(SE_NSE_TT.loc['Science and engineering']))
nse =tuple(list(SE_NSE_TT.loc['Non-science and engineering']))


In [ ]:
radar_chart = pygal.Radar( human_readable=True, legend_at_bottom=True)
radar_chart.title = 'Science & Engineering  and Non-Science & Engineering by Year'
radar_chart.x_labels = [ '2018', '2017', '2016',
       '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007',
       '2006', '2005', '2004', '2003', '2002', '2001', '2000', '1999', '1998',
       '1997', '1996', '1995', '1994', '1993', '1992', '1991', '1990', '1989',
       '1988', '1987', '1986', '1985', '1984', '1983', '1982', '1981', '1980',
       '1979', '1978', '1977', '1976', '1975', '1974', '1973', '1972', '1971',
       '1970', '1969', '1968', '1967', '1966', '1965', '1964', '1963', '1962',
       '1961', '1960', '1959', '1958']
radar_chart.add('Science and engineering',se)
radar_chart.add('Non-science and engineering',nse)

In [ ]:
df1s.head()
LSc   = df1s[df1s['Broad Fields'] == 'Life sciences']
PSES = df1s[df1s['Broad Fields'] == 'Physical sciences and earth sciences']
MCS   = df1s[df1s['Broad Fields'] == 'Mathematics and computer sciences']
PSYC  = df1s[df1s['Broad Fields'] == 'Psychology and social sciences ']
Engr  = df1s[df1s['Broad Fields'] == 'Engineering']
Edu = df1s[df1s['Broad Fields'] == 'Education']
HumAr = df1s[df1s['Broad Fields'] == 'Humanities and arts']
Oth  = df1s[df1s['Broad Fields'] == 'Other']

# SE = df1s[df1s['S&E Fields'] == 'Science and engineering']
# NSE = df1s[df1s['S&E Fields'] != 'Science and engineering']

LSc.head()

In [ ]:
LSc1 = LSc[['Detailed Fields','Number']]
LSc1.head()

In [ ]:
import warnings
warnings.filterwarnings("ignore")
#chmod g-wx,o-wx ~/.python-eggs
fig, axes = plt.subplots(nrows=3, ncols=3)
#LSc1.plot(ax=axes[0], kind = 'bar', grid=True)
LSc1.plot(kind = 'bar', grid=True)


# # fig.suptitle('Bike Rentals at Different Temperatures\nBy Season', fontsize=25,fontweight="bold", color="black", 
# #              position=(0.5,1.01))
# sns.set(style="whitegrid")

# g1 = sns.FacetGrid(LSc, col="Broad Fields", hue_order="Detailed Fields",  height=3, aspect=2)

# g1.map(sns.barplot, "Detailed Fields", "Number");
# g1.set_xticklabels(rotation=15)

# g4 = sns.FacetGrid(LSc, row="Broad Fields", hue_order="Detailed Fields",  height=3, aspect=2)
# g4.map(sns.barplot, "Detailed Fields", "Number");
# g4.set_xticklabels(rotation=15)
# g2= sns.FacetGrid(PSES, col="Broad Fields", hue_order="Detailed Fields",  height=3, aspect=2)
# g2.map(sns.barplot, "Detailed Fields","Number" );
# #g.fig.suptitle("Doctorate Recipients Number of Selected Years by Sex")
# g2.set_xticklabels(rotation=15)

# g3= sns.FacetGrid(MCS, col="Broad Fields", hue_order="Detailed Fields",  height=3, aspect=2)
# g3.map(sns.barplot, "Detailed Fields","Number" );
# #g.fig.suptitle("Doctorate Recipients Number of Selected Years by Sex")
# g3.set_xticklabels(rotation=15)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
#chmod g-wx,o-wx ~/.python-eggs
fig = plt.figure()
plt.subplots_adjust(wspace=0.2, hspace=0.2)

# fig.suptitle('Bike Rentals at Different Temperatures\nBy Season', fontsize=25,fontweight="bold", color="black", 
#              position=(0.5,1.01))
sns.set(style="whitegrid")

g1 = sns.FacetGrid(LSc, col="Broad Fields", hue_order="Detailed Fields",  height=3, aspect=2)

g1.map(sns.barplot, "Detailed Fields", "Number");
g1.set_xticklabels(rotation=15)

g4 = sns.FacetGrid(LSc, row="Broad Fields", hue_order="Detailed Fields",  height=3, aspect=2)
g4.map(sns.barplot, "Detailed Fields", "Number");
g4.set_xticklabels(rotation=15)
g2= sns.FacetGrid(PSES, col="Broad Fields", hue_order="Detailed Fields",  height=3, aspect=2)
g2.map(sns.barplot, "Detailed Fields","Number" );
#g.fig.suptitle("Doctorate Recipients Number of Selected Years by Sex")
g2.set_xticklabels(rotation=15)

g3= sns.FacetGrid(MCS, col="Broad Fields", hue_order="Detailed Fields",  height=3, aspect=2)
g3.map(sns.barplot, "Detailed Fields","Number" );
#g.fig.suptitle("Doctorate Recipients Number of Selected Years by Sex")
g3.set_xticklabels(rotation=15)


In [ ]:
SE = df1s[df1s['S&E Fields'] == 'Science and engineering']
NSE = df1s[df1s['S&E Fields'] != 'Science and engineering']
sns.set(style="whitegrid")
g1 = sns.FacetGrid(SE, col="Broad Fields", hue_order="Detailed Fields",  height=3, aspect=2)

g1.map(sns.barplot,"Detailed Fields", "Number");
g1.set_xticklabels(rotation=15)

In [ ]:
tips = sns.load_dataset("tips")
tips.head()

In [ ]:
#df1s.columns
#SE_NSE1.columns
SE_NSE1=df1s[['Detailed Fields', 'Year', 'Number']]

In [ ]:
SE_NSE2=SE_NSE1.pivot(index='Detailed Fields', columns='Year', values='Number')

In [ ]:
dic = SE_NSE2.to_dict('split')
#SE_NSE2.to_dict()
dic.values()
dic.keys()


In [ ]:
print(dic)
dic2 = ['index', 'data']
[dic[x] for x in dic2]

In [ ]:
dic2[0]

In [ ]:
# dic['data']
#len(dic['data'])
#plt.plot(dic['data'])

In [ ]:
dic['index']
len(dic['index'])

In [ ]:
dic['columns']
len(dic['columns'])

In [ ]:
dic2 = dic['index'] + dic['data']
dic

In [ ]:
for key in dic.keys():
  print(key)

In [ ]:
type(dic['index'])

In [ ]:
df = pd.DataFrame({'col1': [1, 2],
...                    'col2': [0.5, 0.75]},
...                   index=['a', 'b'])
df.columns

In [ ]:
df.head()

In [ ]:
dic= df.to_dict()
dic.keys()

In [ ]:
df.to_dict('split')


In [ ]:

df = SE_NSE1.set_index('Year')
#print(df.T.to_dict(orient='list'))
print(df)

## 2) Doctorate Recipients of S&E and Non-S&E by DemographDoctrate reciepients by Citizenship,  Race/Ethnicity, Sex, S&E Fields, Broad Fields, Detailed Fields, and Year

In [ ]:
# Doctorate Recipients by Demographic Characteristics ---- Data by Citizenship,  Race/Ethnicity, Sex, S&E Fields, Broad Fields, Detailed Fields, and Year
# Data Download from NCSES Interactive Data Tool
df2 = pd.read_csv('../../WorkingFiles/NSF_NCSES/Data/SED_Export_2020-06-17T23_29_53.896Z.csv', skiprows=[0], header=[1])
df2 = df2.set_index(['Citizenship','Race and Ethnicity', 'Sex','S&E Fields', 'Broad Fields', 'Detailed Fields'])
df2.head()

In [ ]:
df2s = df2.stack()
df2s =df2s.to_frame()

df2s = df2s.reset_index()
df2s.columns = ['Citizenship','Race and Ethnicity', 'Sex','S&E Fields', 'Broad Fields', 'Detailed Fields', 'Year', 'Number']
df2s.head()

In [ ]:
df2s = df2s[df2s['Detailed Fields'] != 'Total']
df2s = df2s.reset_index()
df2s = df2s[['Citizenship','Race and Ethnicity', 'Sex','S&E Fields', 'Broad Fields', 'Detailed Fields', 'Year', 'Number']]
df2s.head()